# Run Database
This notebook reads your Strava API run data, extracts the date, time of day, distance (miles), and duration (hours + minutes) of your runs, and loads to a PostgreSQL database. The data can also be uploaded directly from a properly-formatted, 4-column csv.

This notebook requires the following:
 - `strava_functions.py`
 - `pg_functions.py`
 - `analysis_functions.py`
 - json text file with strava api credentials/token + database connection credentials (see `sample-creds.json`)
 

## 1. Function Imports

In [ ]:
from strava_functions import obtain_api_access_info, get_past_runs, generate_runs_df
from pg_functions import generate_db_connection, \
    get_last_logged_run, write_to_postgres, \
    update_postgres, display_db_as_df, \
    perform_sql_query, upload_from_csv
from analysis_functions import generate_weekly_summary
import datetime
import pandas as pd

## 2. Define Database & File info + Variables

- `access_json_path` is the relative path to the file containing your Strava API and Postgres database credentials.<br>
- `my_table' is the table within that database you'll be using.<br>
- the `verbose` parameter will tell you the status of each function that you run.<br>

In [ ]:
access_json_path = "sample-creds.json" #CHANGE THIS!!
my_table = obtain_api_access_info(access_json_path)["postgres"]["table"]
verbose=True

## 3. Getting Started

### 3.1 Initializing your Database (only do this section once)
First, go create a table in pgAdmin if you haven't already. Here's how you can go about doing it:<br>
```
CREATE TABLE IF NOT EXISTS public.runs
    (start_date date,
    start_time time without time zone,
    miles real,
    hours integer,
    minutes integer);
```
Assuming that's all done, let's populate a your new table using `update_postgres(path_to_creds, db_conn_obj, table_name, date_to_update_to, verbose=True, manual_start_date=None)`. By default, this function pulls Strava API from the latest date in the database up to the date specified in the argument. Since you've got an empty table, we'll need to manually specify a starting date and loading up until today.<br>

`connection` is the default variable name used for a psycopg2 Postgres database connection object.<br>
!!! Be sure to close your connection (`connection.close()`) at the end of each cell block. I've included them by default.

In [ ]:
start_date_string = "2022-09-01"
today_datetime_obj = datetime.date.today() #returns a datetime object. We'll eventually turn it to YYYY-mm-dd string format.
today_string = today_datetime_obj.strftime("%Y-%m-%d") #this will be our argument in the function call.

connection = generate_db_connection(access_json_path) #generate that connection object
update_postgres(access_json_path, connection, my_table, today_string, verbose=verbose, manual_start_date=start_date_string)
display_db_as_df(connection, my_table, start_date_string, today_string, verbose=verbose) #let's display it
connection.close() #close your connection object to the database

### 3.2 Updating a Database
Assuming you've done 3.1 already, and you have a database with some data, let's update our database with your most recent runs.

In [ ]:
connection = generate_db_connection(access_json_path)
update_postgres(access_json_path, connection, my_table, today_string, verbose=verbose)
connection.close()

## 4. Weekly Run Stats

Let's generate a Dataframe summarizing your daily and weekly mileage in the below format.

| Week of:  | Mon   | Tues  | Weds  | Thurs | Fri   | Sat   | Sun   | Total |
|---------- |-------|-------|-------|-------|-------|-------|-------|-------|
| mmm dd    |  #    |  #    | #     | #     |  #    | #    | #     | #    |
| mmm dd    |  #    |  #    | #     | #     |  #    | #    | #     | #    |
| mmm dd    |  #    |  #    | #     | #     |  #    | #    | #     | #    |

You can optionally set the display_heatmap boolean to display a heatmap of this dataframe.
To change the colors and thresholds for the heatmap, edit the `generate_weekly_summary()` function within the `analysis_functions.py` document.

In [ ]:
start_date_string = (datetime.date.today() - datetime.timedelta(days=91)).strftime("%Y-%m-%d") #2-months back default
end_date_string = datetime.date.today().strftime("%Y-%m-%d") #default value is current day
connection = generate_db_connection(access_json_path)
display_heatmap = True

weekly_df = generate_weekly_summary(connection, my_table, start_date_string, end_date_string, display_heatmap=display_heatmap, verbose=verbose) 
display(weekly_df) #print the dataframe itself (not the heatmap)
connection.close()